In [1]:
 import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import matplotlib
global df_pred,pred_all
from sklearn.externals import joblib

In [2]:
def predictor_mean(model0_1,modelo_2,X_set):  
    #aplicamos los predict
    forecast_q = model0_1.predict(X_set) 
    forecast = modelo_2.predict(X_set)
   #generamos una lista dummy: 
    forecast_mean = forecast 
    
    for i in range(len(X_set)):
        if  forecast[i] < 135:
            forecast_mean[i] = forecast[i]
        elif  forecast_q[i] > 160:
            forecast_mean[i] = forecast_q[i]
        if  130 < forecast_q[i] <155 and 130 < forecast[i] < 155:
            forecast_mean[i] = (0.3*forecast_q[i] + 0.7*forecast[i])
        if  140 < forecast_q[i] <160 and 140 < forecast[i] < 160:
            forecast_mean[i] = (forecast_q[i] + forecast[i])/2
    return forecast_mean

In [3]:
target1 = "PM10mean_y_frcst_"
conta1 = 'PM10'
target2 = "O3_y_frcst_"
conta2 = 'O3'
target = "O3_y_frcst_"
conta = 'O3'


In [4]:
def prediccion(target,conta):

    #cargamos el DF
    df_predict = pd.read_csv('./datos/ingesta_modelos_'+ conta +'.csv') 

    df_fechas = df_predict["fecha"]
    df_predict.drop(["fecha"],inplace=True,axis=1)

    
    #cargamos los modelos
    model = [0 for col in range(25)]
    modelq = [0 for col in range(25)]
    for i in range(0,24):
        model[i] = joblib.load(str('/home/paw/DanielBustillos/contaminación/pipeline-norberto/modelos/'+conta+'/'+
                                   conta+'_'+str(i+1)+'hora.pkl'))
        modelq[i] = joblib.load(str('/home/paw/DanielBustillos/contaminación/pipeline-norberto/modelos/'+conta+'/'+
                                    conta+'_'+str(i+1)+'hora_quantile.pkl'))
        
    #hacemos el predict

    for i in range(0,24):
        df_predict[target + str(i+1)] = predictor_mean(modelq[i],model[i],df_predict)

        
    
    df_predict[target+'24'].head(1)
    
    pred = []
    for i in range(1,25):
        pred.append(df_predict[str(target+str(i))].head(1).values[0])

    df_predict["fecha"] = df_fechas

    ## predict de las últimas 12 horas
    df_predict_historico = df_predict[["fecha",target+'2']].copy()

    
    if conta == "PM10":
        df_predict_historico["fecha"] = df_predict_historico["fecha"].shift(0)
    elif conta == "O3":
        df_predict_historico["fecha"] = df_predict_historico["fecha"].shift(12)
    else: print("ERROR")
        

    df_predict_historico = df_predict_historico[["fecha",target+'6']].dropna()#.iloc[12:,:]#.head(13)
    df_predict_historico.rename(columns={target+'24': conta+'_pron_a_12hrs'}, inplace=True)
    
    
    
    #predict 12 horas
    df_pred =  pd.DataFrame(columns=["pred_0"])
    for i in range(0,24):
        df_pred[str("pred_"+str(i))] = [pred[i]]

    for i in range(0,24):
        df_pred.loc[i,conta+'_pronóstico'] = df_pred.iloc[0,i]
    for i in range(0,24):
        df_pred.loc[i,'fecha'] = pd.date_range(df_fechas[0], periods=25, freq='H').tolist()[i+1]
        df_pred = df_pred[[conta+"_pronóstico","fecha"]]
            
    if conta == "PM10":
        df_predict = df_predict[["fecha","PM10","PM10_y","PM10_x","PM10mean_y"]]
    elif conta == "O3":
        df_predict = df_predict[["fecha",'TMP_x','TMP_y',
                                            'O3_y','O3','O3_x','RH','TMP']]
    else: print("ERROR")
        
        
    df_pred['fecha'] = df_pred['fecha'].astype(str)
    
    
    return  df_predict_historico,df_predict,df_pred

Ejecutamos la función:

In [5]:
#df_predict_historico_O3,df_predict_O3,df_pred_O3 = prediccion(target2,conta2)
df_predict_historico_PM10,df_predict_PM10,df_pred_PM10 = prediccion(target1,conta1)

FileNotFoundError: [Errno 2] File b'./datos/ingesta_modelos_PM10.csv' does not exist: b'./datos/ingesta_modelos_PM10.csv'

In [ ]:
df_pred_O3.head(24)

In [ ]:
df_export = pd.concat([df_predict_O3.set_index('fecha'),df_predict_PM10.set_index('fecha')], axis=1, join='inner').reset_index()

In [ ]:
df_export.head(5)

In [ ]:
df_pron_hist = pd.concat([df_predict_historico_O3.set_index('fecha'),df_predict_historico_PM10.set_index('fecha')], axis=1, join='inner').reset_index()

In [ ]:
df_export = pd.merge(df_pron_hist, df_export, on='fecha',how="inner")

In [ ]:
df_pronosticados = pd.concat([df_pred_O3.set_index('fecha'),df_pred_PM10.set_index('fecha')], axis=1, join='inner').reset_index()

In [ ]:
#df_export =
df_export = pd.merge(df_pronosticados, df_export, on='fecha',how="outer")

Unimos los DF:

In [ ]:
df_export['fecha'] =  pd.to_datetime(df_export['fecha'], format='%Y-%m-%d %H:%M')
df_export.PM10mean_y_frcst_12 = df_export.PM10mean_y_frcst_12-10
df_export.loc[23,"O3_pronóstico"] = df_export.loc[23,"O3_y_frcst_12"]
df_export.loc[23,"PM10_pronóstico"] = df_export.loc[23,"PM10mean_y_frcst_12"]
df_export.PM10_pronóstico = df_export.PM10_pronóstico

In [ ]:
df_export.to_csv("./datos/datos_pau.csv",sep=',', encoding='utf-8',index=False)

In [ ]:
fig = plt.figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
ax = fig.gca()

# df_export.tail(60).head(50).plot(x="fecha", y=[ "O3_y"],ax=ax,label=["O3 Medido"],color="dodgerblue")
# df_export.tail(60).head(100).plot(x="fecha", y=["O3_y_frcst_6"],ax=ax,label=["Pronóstico O3 a 12 horas"],color='darkblue',dashes=[6, 2])

df_export.plot(x="fecha", y=[ "PM10mean_y"],ax=ax,label=["PM10 mean Medido"],color="saddlebrown")
df_export.plot(x="fecha", y=[ "PM10_pronóstico"],ax=ax,label=["PM10 pronostico"],color="blue")
df_export.plot(x="fecha", y=["PM10mean_y_frcst_12"],ax=ax,label=["Pronóstico PM10 a 12 horas"],color='maroon',dashes=[6, 2])

plt.ylabel("O3[mg/m3]")
plt.title("Comparación del Modelo")


# And a corresponding grid
ax.grid(which='both')

# Or if you want different settings for the grids:
ax.grid(which='minor', alpha=0.01)
ax.grid(which='major', alpha=0.3)

# plt.savefig('/home/paw/DanielBustillos/contaminación/imagenes/o3_comparacion.png', bbox_inches='tight'
#             , dpi=400)
plt.legend()
plt.show()

In [ ]:
fig = plt.figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
ax = fig.gca()

# df_export.tail(60).head(50).plot(x="fecha", y=[ "O3_y"],ax=ax,label=["O3 Medido"],color="dodgerblue")
# df_export.tail(60).head(100).plot(x="fecha", y=["O3_y_frcst_6"],ax=ax,label=["Pronóstico O3 a 12 horas"],color='darkblue',dashes=[6, 2])

df_export.plot(x="fecha", y=[ "O3_y"],ax=ax,label=["O3 Medido"],color="royalblue")
df_export.plot(x="fecha", y=[ "O3_pronóstico"],ax=ax,label=["O3 pronostico"],color="blue",dashes=[6, 2])
df_export.plot(x="fecha", y=["O3_y_frcst_12"],ax=ax,label=["Pronóstico O3 a 12 horas"],color='darkblue')

plt.ylabel("O3[mg/m3]")
plt.title("Comparación del Modelo")


# And a corresponding grid
ax.grid(which='both')

# Or if you want different settings for the grids:
ax.grid(which='minor', alpha=0.01)
ax.grid(which='major', alpha=0.3)

# plt.savefig('/home/paw/DanielBustillos/contaminación/imagenes/o3_comparacion.png', bbox_inches='tight'
#             , dpi=400)
plt.legend()
plt.show()